# Banka - prijava

Na bazi za našo aplikacijo bomo uporabnikom omogočili prijavo. V ta namen jim bo potrebno dodati uporabniška imena in gesla.

In [1]:
from bottleext import baza
import bcrypt

cur = baza.cursor()

PRAGMA foreign_keys = ON;


In [2]:
cur.execute("ALTER TABLE oseba ADD COLUMN up_ime TEXT") # SQLite ne dovoli dodajanje stolpca z določilom UNIQUE
cur.execute("ALTER TABLE oseba ADD COLUMN geslo BLOB")
cur.execute("ALTER TABLE oseba ADD COLUMN admin INTEGER NOT NULL DEFAULT 0")

ALTER TABLE oseba ADD COLUMN up_ime TEXT
ALTER TABLE oseba ADD COLUMN geslo BLOB
ALTER TABLE oseba ADD COLUMN admin INTEGER NOT NULL DEFAULT 0


Za ustvarjanje in preverjanje gesel bomo uporabili knjižnico `bcrypt`. Napišimo funkcijo, ki pretvori dano geslo v obliko, kakršno bomo hranili v bazi - ta sestoji iz naključno generirane *soli* in *zgostitve* (*hash*) te soli skupaj z danim geslom. Z drugo funkcijo bomo potem preverjali veljavnost gesla.

In [3]:
def nastavi_geslo(geslo):
    geslo = geslo.encode("utf-8")
    sol = bcrypt.gensalt()
    return bcrypt.hashpw(geslo, sol)

def preveri_geslo(geslo, zgostitev):
    geslo = geslo.encode("utf-8")
    return bcrypt.checkpw(geslo, zgostitev)

Poskusimo z vzorčnim geslom.

In [4]:
(zgostitev := nastavi_geslo('geslo'))

b'$2b$12$e/zub.KZ3pllLYJHHWXdU.9F/1DL5McFVVwH//VTaqSNz7ZX1gv6q'

In [5]:
preveri_geslo('geslo', zgostitev)

True

In [6]:
preveri_geslo('nekajdrugega', zgostitev)

False

Funkcijo `nastavi_geslo` bomo registrirali na povezavi na podatkovno bazo, da jo bomo lahko uporabili v poizvedbah.

In [7]:
baza.create_function('nastavi_geslo', 1, nastavi_geslo)

Preizkusimo našo funkcijo.

In [8]:
cur.execute("SELECT nastavi_geslo(?)", ("geslo", ))
cur.fetchone()

SELECT nastavi_geslo('geslo')


(b'$2b$12$0qn6gZwJvgkX6TCuk7/s5eXKrOf0V6.Yth7TT.bztw2cSssXNeHQa',)

Sedaj lahko uporabnikom nastavimo uporabniška imena in gesla.

In [9]:
with baza:
    cur.execute("""
        UPDATE oseba SET
            up_ime = lower(substr(ime, 1, 1) || priimek || substr(emso, -1, 1)),
            geslo = nastavi_geslo(ime)
    """)

BEGIN 

        UPDATE oseba SET
            up_ime = lower(substr(ime, 1, 1) || priimek || substr(emso, -1, 1)),
            geslo = nastavi_geslo(ime)
            LIMIT 1
    
COMMIT
